---
  title: "RestFull 03: Représentation des données"
  description: "Introduction to RestFull Web Services in Java with Jakarta RESTful Web Services."
  categories: 
    - Java
    - Lecture
    - RestFull
  provide_notes: true
  provide_slides: false
  jupyter: java-lts    
  echo: true
  output: true    
---

{{< embed ./quarto-utils/_version.qmd >}}

Les resources sont habituellement échangées en utilisant des langage de description standards comme [XML](https://www.w3.org/TR/xml11/) ou [JSON](https://www.ecma-international.org/publications-and-standards/standards/ecma-404/) (pour être précis JSON n'est pas vraiment un "standard" du web). Il est donc très courant de convertir des données vers et depuis Java. Pour cela, des API standards existent.

## JAXB
Nous allons voir maintenant une introduction rapide au mapping XML<->Java. La définition des formats de données XML se fait par annotation des entités en utilisant le standard [JAXB](https://jakarta.ee/specifications/xml-binding/3.0/jakarta-xml-binding-spec-3.0.html) (Java Architecture for XML Binding) : `@XmlElement`, `@XmlType`, `@XmlAttribute`, `@XmlTransient`, `@XmlValue`, ...

Depuis Java 9, il est nécessaire d'ajouter les dépendances suivantes pour traiter des données XML avec JAXB : 

In [1]:
%%loadFromPOM
<dependency>
  <groupId>jakarta.xml.bind</groupId> 
  <artifactId>jakarta.xml.bind-api</artifactId>
  <version>3.0.0</version>
</dependency>
<dependency>
  <groupId>org.glassfish.jaxb</groupId>
  <artifactId>jaxb-runtime</artifactId>
  <version>3.0.0</version>
</dependency>

SLF4J(I): Connected with provider of type [ch.qos.logback.classic.spi.LogbackServiceProvider]


La classe `Task` ci-dessous est un exemple simple. marquée comme étant représentée comme un élément XML (`@XmlRootElement`). On précise que les annotations sont faites sur les champs avec `@XmlAccessorType` (utile avec Lombok).

Par défaut, les propriétés sont représentées comme des éléments XML. Il est possible de préciser que l'on veut un attribut (`@XmlAttribute`) sur `id`, de contrôler leur nom (paramètre `name`) et de définir ceux qui ne doivent pas apparaitre (`@XMLTransient`). Il est aussi possible de contrôler l'ordre d'apparition des éléments (`propOrder` de `@XmlType`).

Attention, un constructeur sans paramètre (au maximum `protected`) est obligatoire (pour permettre la reconstruction). Sinon `@XmlType.factoryMethod()` et `@XmlType.factoryClass()` permettent d'utiliser une factory s'il s'agit d'une méthode statique sans paramètre.

Dans le cas d'une collection `@XmlElementWrapper` permet d'ajouter un élément parent au contenu et `@XmlElements` contrôle le type des éléments en fonction du type réel Java. Pour des primitifs `@XmlList` permet de générer des listes avec un espace comme séparateur.

`@XmlType` est similaire à `@XmlRootElement` si la classe ne doit apparitre que comme un sous-élément. 

`@XmlValue` ne peut être utilisée que sur une seule propriété dont la valeur sera alors le contenu de l'élément (sans élément parent). 

In [2]:
//| output: false
//| echo: false
import jakarta.xml.bind.annotation.*;
import java.util.List;
import java.util.ArrayList;

In [3]:
@XmlRootElement(name="task", namespace="http://bruno.univ-tln.fr/sample-jaxb/task")
@XmlType(propOrder = { "id", "state", "title", "description", "tags"})
@XmlAccessorType(XmlAccessType.FIELD)
public class Task {
   @XmlAttribute(name="id")
   private long id=-1;

   private String title;

   @XmlElement(name="status")
   private State state=State.OPENED;
   
   @XmlTransient
   private int age = -1;
        
   @XmlElementWrapper(name="tags")
   @XmlElements({@XmlElement(name="tag",type=String.class)})
   private List<String> tags; 
    
   private Description description = new Description(); 
    
   protected Task() {} 
   public Task(long id, String title, State state, List<String> tags) {
       this.id=id; this.title=title; this.state=state; this.tags = tags;
       }  
    
   public String toString() {return "Task {id="+id+",title='"+title+"',status='"+state+", tags='"+tags+"', description='"+description+"'}";}
    
 @XmlEnum(Integer.class)   
 //Par défaut vers String (donc @XmlEnumValue inutile)
 //@XmlEnum(String.class)   
 public enum State {
    @XmlEnumValue("1") OPENED, 
    @XmlEnumValue("0") CLOSED
 }
 
 @XmlAccessorType(XmlAccessType.FIELD)    
 public static class Description {
     @XmlValue 
     private String content = "...";
     public String toString() {return content;}
 }
    
}

JAXB offre la classe [JAXBContext](https://jakarta.ee/specifications/xml-binding/3.0/jakarta-xml-binding-spec-3.0.html#jaxbcontext) pour transformer une classe Java en XML (Marshalling).

In [4]:
//| output: false
//| echo: false
import jakarta.xml.bind.JAXBContext;
import jakarta.xml.bind.Marshaller;

In [5]:
Task task = new Task(1L, "First task", Task.State.OPENED, Arrays.asList("important","outside"));

//Création du contexte JAXB sur la classe Task
JAXBContext jaxbContext = JAXBContext.newInstance(Task.class);

//Création de la classe qui converti vers XML
Marshaller marshaller = jaxbContext.createMarshaller();
//Définition des paramètres de la conversion (optionnel)
marshaller.setProperty(Marshaller.JAXB_FORMATTED_OUTPUT, true);
//Exécution de la conversion
StringWriter sw = new StringWriter();
marshaller.marshal(task, sw);
String result=sw.toString();

In [6]:
//| output: true
//| echo: false

render("```xml\n"+result+"\n```", "text/markdown");

```xml
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<ns2:task id="1" xmlns:ns2="http://bruno.univ-tln.fr/sample-jaxb/task">
    <status>1</status>
    <title>First task</title>
    <description>...</description>
    <tags>
        <tag>important</tag>
        <tag>outside</tag>
    </tags>
</ns2:task>

```

Le contexte JABX permet aussi simplement de réaliser l'opération inverse (UnMarshalling) à partir d'un document XML contenu dans une String, un fichier, d'un flux, ...

In [7]:
String xmlString="""
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<ns2:task xmlns:ns2="http://bruno.univ-tln.fr/sample-jaxb/task" id="2">
    <status>0</status>
    <title>Another task</title>
    <description>Une tache inutile</description>
    <tags>
        <tag>spare-time</tag>
        <tag>fun</tag>
    </tags>
</ns2:task>""";
//Conversion d'n document XML 
jaxbContext.createUnmarshaller()
       .unmarshal(new StringReader(xmlString));

Task {id=2,title='Another task',status='CLOSED, tags='[spare-time, fun]', description='Une tache inutile'}

Dans le cas de JAX-RS, c'est le framework qui prend en charge la transformation des données retournées et reçues à condition d'ajouter la dépendance suivante en plus de celles de JAXB :

```xml
<dependency>
 <groupId>org.glassfish.jersey.media</groupId>
 <artifactId>jersey-media-jaxb</artifactId>
</dependency>
```

JAXB Permet aussi de générer automatique le Schema XML à partir des classes Java. Il suffit d'écrire une sous-classe de `SchemaOutputResolver` pour indiquer où les résultat doit être produit. Ci dessous deux exemples pour obtenir une String et des fichiers. 

In [8]:
//| output: false
//| echo: false
import jakarta.xml.bind.SchemaOutputResolver;
import javax.xml.transform.Result;
import javax.xml.transform.stream.StreamResult;

In [9]:
public class StringSchemaOutputResolver extends SchemaOutputResolver {
    private StringWriter stringWriter = new StringWriter();    

    public Result createOutput(String namespaceURI, String suggestedFileName) throws IOException  {
        StreamResult result = new StreamResult(stringWriter);
        result.setSystemId(suggestedFileName);
        return result;
    }

    public String getSchema() {
        return stringWriter.toString();
    }

}


public class FileSchemaOutputResolver extends SchemaOutputResolver {        
    @Override
    public Result createOutput(String nameSpaceURI, String suggestedName) throws IOException {
        System.out.println(nameSpaceURI+" "+suggestedName);
        StreamResult streamResult = new StreamResult(suggestedName);
        return streamResult;
    }
}

StringSchemaOutputResolver stringSchemaOutputResolver = new StringSchemaOutputResolver();
jaxbContext.generateSchema(stringSchemaOutputResolver);
String result = stringSchemaOutputResolver.getSchema();


In [10]:
//| output: true
//| echo: false
render("```xml\n"+result+"\n```", "text/markdown");

```xml
<?xml version="1.0" standalone="yes"?>
<xs:schema version="1.0" xmlns:xs="http://www.w3.org/2001/XMLSchema">

  <xs:complexType name="task">
    <xs:sequence>
      <xs:element name="status" type="state" minOccurs="0"/>
      <xs:element name="title" type="xs:string" minOccurs="0"/>
      <xs:element name="description" type="description" minOccurs="0"/>
      <xs:element name="tags" minOccurs="0">
        <xs:complexType>
          <xs:sequence>
            <xs:element name="tag" type="xs:string" minOccurs="0" maxOccurs="unbounded"/>
          </xs:sequence>
        </xs:complexType>
      </xs:element>
    </xs:sequence>
    <xs:attribute name="id" type="xs:long" use="required"/>
  </xs:complexType>

  <xs:simpleType name="description">
    <xs:restriction base="xs:string"/>
  </xs:simpleType>

  <xs:simpleType name="state">
    <xs:restriction base="xs:int">
      <xs:enumeration value="1"/>
      <xs:enumeration value="0"/>
    </xs:restriction>
  </xs:simpleType>
</xs:schema>

<?xml version="1.0" standalone="yes"?>
<xs:schema version="1.0" targetNamespace="http://bruno.univ-tln.fr/sample-jaxb/task" xmlns:xs="http://www.w3.org/2001/XMLSchema">

  <xs:import schemaLocation="schema2.xsd"/>

  <xs:element name="task" type="task"/>

</xs:schema>


```

## JSON
Le standard officiel pour JSON est maintenant [JSON-B](http://json-b.net/) (Java API for JSON Binding). Cependant, des fonctionnalités importantes sont manquantes comme la gestion des types Polymorphes ou de certaines classes importantes en natif (comme les collections eclipses). Nous utiliserons donc une autre librairie : [Jackson](https://github.com/FasterXML/jackson) (cf. pom.xml). 

Pour l'utiliser, il suffit d'ajouter les dépendances suivantes :

In [11]:
%%pom
<!-- -->
<dependency>
 <groupId>com.fasterxml.jackson.core</groupId>
 <artifactId>jackson-databind</artifactId>
 <version>2.16.1</version>
</dependency>

<!-- Optionnel ajoute le support de type Java8 hors Date -->
<dependency>
    <groupId>com.fasterxml.jackson.datatype</groupId>
    <artifactId>jackson-datatype-jdk8</artifactId>
    <version>2.16.1</version>
    <type>pom</type>
</dependency>
     
<!-- Optionnel ajoute le support des Date Java8 -->
<dependency>
    <groupId>com.fasterxml.jackson.datatype</groupId>
    <artifactId>jackson-datatype-jsr310</artifactId>
    <version>2.16.1</version>
</dependency>

Le contrôle de la sérialisation/désérialisation se fait principalement par annotation des entités. Les méthodes de sérialisation/désérialisations sont alors générées automatiquement. Il est possible de créer manuellement ces méthodes pour un contrôle plus précis. 

Les principales annotations sont données dans le tableau ci dessous et illustrées dans l'exemple suivant. 

|annotation|Description|
|---|---|
|`@JsonProperty` |contrôle le nom d'une propriété.|
|`@JsonRootName` |défini les nom d'un élément "wrapper", doit être activée dans l'objectMapper.|
|`@JsonPropertyOrder` |défini l'ordre des propriétés. |
|`@JsonRawValue` |indique qu'une propriété contient du JSON et doit être utilisée sans conversion.|
|`@JsonValue` |indique la seule méthode qui retourne le contenu à serialiser.  Nécessite un constructeur avec un paramètre du même type pour la désérialisation.|
|`@JsonIgnore` (sur une propriété) |pour ignorer une ou plusieurs propriété.|
|`@JsonIgnoreProperties` (sur la classe) |pour ignorer une ou plusieurs propriété. |
|`@JsonIgnoreType` (sur la classe) | permet d'ignorer toutes les propriétés d'un type donné.|
|`@JsonUnwrapped` | inclut directement les propriétés d'un objet dans la classe qui le référence. |
|`@JsonInclude` |permet d'inclure ou d'ignorer les propriétés dont la valeur est nulle, vide ou celle par défaut.|


Pour les types polymorphes des annotations spécifiques pour les classes qui permettent d'indiquer comment le type est indiqué (`@JsonTypeInfo`), quels sont les sous-types (`@JsonSubTypes`) et le nom donné à chaque type (`@JsonTypeName`).

`@JsonView(XXX.class)` définit des vues différentes qui peuvent être choisir dans l'objectmapper.


In [12]:
//| output: false
//| echo: false
import java.util.List;
import java.time.LocalDateTime;
import java.util.ArrayList;
import com.fasterxml.jackson.annotation.*;  

In [13]:
public class View {
    public static class Minimal {}
    public static class Complete extends Minimal {}
}

@JsonTypeInfo(include = JsonTypeInfo.As.WRAPPER_OBJECT, use = JsonTypeInfo.Id.NAME)
@JsonTypeName("Task")
@JsonPropertyOrder({ "id", "state", "title", "description", "tags"})
@JsonIgnoreProperties({"age"})
@JsonInclude(JsonInclude.Include.NON_EMPTY)
public class Task {
   private long id=-1;

   private String title;

   @JsonProperty("status")
   private State state=State.OPENED;
   
   @JsonIgnore
   private int age = -1;
        
   private List<String> tags; 
 
   @JsonView(View.Complete.class)
   private Description description = new Description(""); 
    
   protected Task() {} 
   public Task(long id, String title, State state, List<String> tags) {
       this.id=id; this.title=title; this.state=state; this.tags = tags;
       }  
    
   @JsonFormat(
      shape = JsonFormat.Shape.STRING,
      pattern = "yyyy-MM-dd@HH:mm:ss")
   public LocalDateTime creationDate =  LocalDateTime.now();
    
   public String toString() {return "Task {id="+id+",title='"+title+"',status='"+state+", tags='"+tags+"', description='"+description+"'}";}
    
 public enum State {
    OPENED, 
    CLOSED
 }
 
//@JsonTypeInfo(include = JsonTypeInfo.As.WRAPPER_OBJECT, use = JsonTypeInfo.Id.NAME)
@JsonTypeName("Description")
@JsonInclude(JsonInclude.Include.NON_EMPTY)
public static class Description {
     @JsonValue 
     private String content;
     public String toString() {return content;}
     public Description(String content) {
         this.content=content;
     }
 }
    
}

La sérialisation/désérialisation est réalisée à l'aide d'une classe appelée ObjectMapper. Dans le cas de JAX-RS cette opération sera réalisée automatiquement par le framework.

In [14]:
//| output: false
//| echo: false
import com.fasterxml.jackson.databind.ObjectMapper;
import com.fasterxml.jackson.annotation.JsonAutoDetect.Visibility;
import com.fasterxml.jackson.databind.SerializationFeature;

In [15]:
Task[] tasks = {new Task(1L, "First task", Task.State.OPENED, Arrays.asList("important","outside")),
            new Task(2L, "Second task", Task.State.OPENED, Arrays.asList("optionnal"))};

ObjectMapper objectMapper = new ObjectMapper()
    
    //Only needed because task doesn't have getters.
    .setVisibility(PropertyAccessor.FIELD, Visibility.ANY)

    //Optionnal : wrap elements (see @JsonRootName)
    //.enable(SerializationFeature.WRAP_ROOT_VALUE)

    //Optionnal : pretty print the result.
    .enable(SerializationFeature.INDENT_OUTPUT)

    //Register jackson modules like date 
    .findAndRegisterModules();

    //.writerWithView(View.Minimal.class);
    //.writerWithView(View.Complete.class);


String taskAsString = 
    objectMapper
      //Choose or view (or none)
    //.writerWithView(View.Minimal.class)
      //.writerWithView(View.Complete.class)
    //writes to a file  
      //.writeValue(new File("task.json"), tasks)
    //or returns a String
      .writeValueAsString(tasks);


In [16]:
//| output: true
//| echo: false
//Render the formatted result in the notebook
render("```json\n"+taskAsString+"\n```", "text/markdown");

```json
[ {
  "Task" : {
    "id" : 1,
    "status" : "OPENED",
    "title" : "First task",
    "tags" : [ "important", "outside" ],
    "creationDate" : "2024-10-03@13:06:56"
  }
}, {
  "Task" : {
    "id" : 2,
    "status" : "OPENED",
    "title" : "Second task",
    "tags" : [ "optionnal" ],
    "creationDate" : "2024-10-03@13:06:56"
  }
} ]
```

La construction d'une object Java depuis JSON est très simple avec la methode `readValue` de `ObjectMapper`.

In [17]:
String jsonString="""
{
  "Task" : {
    "id" : 3,
    "status" : "CLOSED",
    "title" : "Another task",
    "description" : "inutile",
    "tags" : [ "spare-time", "fun"],
    "creationDate" : "2021-02-23@04:06:37"
  }
}    
""";
    
objectMapper.readValue(jsonString, Task.class);

Task {id=3,title='Another task',status='CLOSED, tags='[spare-time, fun]', description='inutile'}

Jackson propose une gestion simple des références qui prend en compte les cycles. Dans notre exemple, si une tâche est associée à un utilisateur qui référence aussi toutes ses tâches, il y a une boucle infinie lors de la sérialisation. Une solution consiste à utiliser dans au moins l'un des deux l'indentifiant de l'autre.

L'annotation `@JsonIdentityInfo` permet de définir la solution pour identifier les instances d'une classe.  Elle est alors utilisée automatiquement quand cela est nécessaire.

`@JsonIdentityReference(alwaysAsId = true)` permet de contrôler l'usage de l'identifiant (ici de le rendre systématique).

`@JsonBackReference` et `@JsonManagedReference` pour les références unidirectionnelles.

In [18]:
//| output: false
//| echo: false
//BEGIN NOT NEED IN REAL JAVA
//Temporary User & Task class déclaration because forward declaration in JShell does not work with annotations
public class User{};
public class Task{};
//END OF NOT NEED IN REAL JAVA    


In [19]:
@JsonTypeName(value = "Data")
@JsonIdentityInfo(
  generator = ObjectIdGenerators.PropertyGenerator.class,
  property = "uuid")

@JsonTypeInfo(//use = JsonTypeInfo.Id.CLASS, //Use the class
              use = JsonTypeInfo.Id.NAME,  //or use the name
              //property = "@class",
              include = JsonTypeInfo.As.PROPERTY            
              )
@JsonSubTypes({
        @JsonSubTypes.Type(value = User.class, name = "User"),
        @JsonSubTypes.Type(value = Task.class, name = "Task")
    })
public class Data {
    protected UUID uuid = UUID.randomUUID();   
    public String toString() {return "User {uuid="+uuid+"'}";}    
}

In [20]:
@JsonTypeName(value = "User")
public class User extends Data {
    private String name;
    
    @JsonIdentityReference(alwaysAsId = true)    
    private List<Task> tasks = new ArrayList<>();
    public void addTask(Task task) {tasks.add(task);}
    public List<Task> getTasks() {return tasks;}
    
    protected User() {};
    public User(String name) {this.name = name;}
    public String toString() {return "User {uuid="+uuid+",name='"+name+"'}";}    
}

In [21]:
@JsonTypeName(value = "Task")
public class Task extends Data {
    private String title;
    
    @JsonIdentityReference(alwaysAsId = true)    
    private User owner;
    
    protected Task() {};
    @JsonCreator
    public Task(@JsonProperty("title") String title, @JsonProperty("owner") User owner) {        
        this.title = title; this.owner = owner;
        owner.addTask(this);
        //System.out.println("'Task Constructor Called: ' "+title+" "+owner+" "+owner.getTasks());
    }
    public void setOwner(User owner) {
        this.owner = owner;
        owner.addTask(this);
    }
    
    public String toString() {return "Task {uuid="+uuid+",title='"+title+"'}";}
}

In [22]:
User user1 = new User("John");
Task task1 = new Task("T1",user1);
Task task2 = new Task("T2",user1);

List<Data> dataList = Arrays.asList(user1, task1, task2); 
Data[] dataArray = new Data[]{user1, task1, task2}; 

ObjectMapper objectMapper = new ObjectMapper()
    .setVisibility(PropertyAccessor.FIELD, Visibility.ANY)
    .enable(SerializationFeature.INDENT_OUTPUT)
    .findAndRegisterModules();

String result = objectMapper.writeValueAsString(dataArray);

In [23]:
//| output: true
//| echo: false
render("```json\n"+result+"\n```", "text/markdown");

```json
[ {
  "@type" : "User",
  "uuid" : "ed955c71-ba2e-44eb-94cb-82e960970fca",
  "name" : "John",
  "tasks" : [ "f3c8d563-3573-4dbc-be7c-1a74c7642070", "faafcdfd-ad8d-431c-a678-0fdf8322a452" ]
}, {
  "@type" : "Task",
  "uuid" : "f3c8d563-3573-4dbc-be7c-1a74c7642070",
  "title" : "T1",
  "owner" : "ed955c71-ba2e-44eb-94cb-82e960970fca"
}, {
  "@type" : "Task",
  "uuid" : "faafcdfd-ad8d-431c-a678-0fdf8322a452",
  "title" : "T2",
  "owner" : "ed955c71-ba2e-44eb-94cb-82e960970fca"
} ]
```

La lecture de données JSON se fait de la même manière. Attention, pour le lien bidirectionnel il ne doit pas apparaitre deux fois (dans User et dans Task) sinon les données sont ajoutées en double.

In [24]:
//We read the JSON String to produce Java Objects
Data[] data2 = objectMapper.readValue("""
[ {
  "@type" : "User",
  "uuid" : "487d6096-7608-11eb-9439-0242ac130002",
  "name" : "Mary"
}, {
  "@type" : "Task",
  "uuid" : "5ba90634-7608-11eb-9439-0242ac130002",
  "title" : "TM1",
  "owner" : "487d6096-7608-11eb-9439-0242ac130002"
}, {
  "@type" : "Task",
  "uuid" : "69b64aa2-7608-11eb-9439-0242ac130002",
  "title" : "TM2",
  "owner" : "487d6096-7608-11eb-9439-0242ac130002"
} ]""", Data[].class);

//We produce JSON from the generated Java objects.                                      
String result = objectMapper.writeValueAsString(data2);

In [25]:
//| output: true
//| echo: false                          
render("```json\n"+result+"\n```", "text/markdown");                               

```json
[ {
  "@type" : "User",
  "uuid" : "487d6096-7608-11eb-9439-0242ac130002",
  "name" : "Mary",
  "tasks" : [ "5ba90634-7608-11eb-9439-0242ac130002", "69b64aa2-7608-11eb-9439-0242ac130002" ]
}, {
  "@type" : "Task",
  "uuid" : "5ba90634-7608-11eb-9439-0242ac130002",
  "title" : "TM1",
  "owner" : "487d6096-7608-11eb-9439-0242ac130002"
}, {
  "@type" : "Task",
  "uuid" : "69b64aa2-7608-11eb-9439-0242ac130002",
  "title" : "TM2",
  "owner" : "487d6096-7608-11eb-9439-0242ac130002"
} ]
```

Pour aller plus loin, JSON n'est en fait pas un standard du Web et donc chaque format est "propriétaire". [JSON-LD](https://json-ld.org/) qui s'appuie sur JSON pour représenter des données sémantiques sur le Web est une meilleure solution. Pour être, complètement compatible avec l'approche HATEOS, Un vocabulaire spécifique pour les API RESGT construit au dessus de JSON-LD appelé [Hydra](http://www.hydra-cg.com/spec/latest/core/) est en cours de définition. LE site [schemas.org](https://schema.org/docs/schemas.html) propose de standardiser des schémas courants.

## Autres formats
Par curiosité, Jackson propose aussi d'autres formats comme YAML.

In [26]:
%%loadFromPOM
<dependency>
 <groupId>com.fasterxml.jackson.dataformat</groupId>
 <artifactId>jackson-dataformat-yaml</artifactId>
 <version>2.16.1</version>
</dependency>

In [27]:
//| output: false
//| echo: false  
import com.fasterxml.jackson.dataformat.yaml.YAMLFactory;

In [28]:
ObjectMapper objectMapperYAML = new ObjectMapper(new YAMLFactory())
    .setVisibility(PropertyAccessor.FIELD, Visibility.ANY)
    .enable(SerializationFeature.INDENT_OUTPUT)
    .findAndRegisterModules();

String result = objectMapperYAML.writeValueAsString(data2);


In [29]:
//| output: true
//| echo: false  
render("```yaml\n"+result+"\n```", "text/markdown");

```yaml
---
- !<User>
  &487d6096-7608-11eb-9439-0242ac130002 uuid: "487d6096-7608-11eb-9439-0242ac130002"
  name: "Mary"
  tasks:
  - "5ba90634-7608-11eb-9439-0242ac130002"
  - "69b64aa2-7608-11eb-9439-0242ac130002"
- !<Task>
  &5ba90634-7608-11eb-9439-0242ac130002 uuid: "5ba90634-7608-11eb-9439-0242ac130002"
  title: "TM1"
  owner: *487d6096-7608-11eb-9439-0242ac130002
- !<Task>
  &69b64aa2-7608-11eb-9439-0242ac130002 uuid: "69b64aa2-7608-11eb-9439-0242ac130002"
  title: "TM2"
  owner: *487d6096-7608-11eb-9439-0242ac130002

```